In [ ]:
# Import neccesary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Model
from keras.layers import Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [ ]:
# Train the model

model = MobileNet(input_shape=(224,224,3), include_top=False)

for layer in model.layers:
    layer.trainable = False

X = Flatten()(model.output)
X = Dense(units= 7, activation='softmax')(X)

model = Model(model.input, X)
model.summary()


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data Preprocessing

# Preprocessing Train Data

train_datagen = ImageDataGenerator(
    zoom_range = 0.2,
    shear_range= 0.2,
    horizontal_flip=True,
    rescale=1./255
)

train_data = train_datagen.flow_from_directory(
    directory='../FacialAnalysis/TrainData/',
    target_size=(224,224),
    batch_size=32
)

print(train_data.class_indices)


# Preprocessing Validation Data

val_datagen = ImageDataGenerator(rescale=1./255)

val_data = val_datagen.flow_from_directory(
    directory='../FacialAnalysis/TestData/',
    target_size=(224,224),
    batch_size=32
)

print(val_data.class_indices)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_data = val_datagen.flow_from_directory(
    directory='../FacialAnalysis/TestData/',
    target_size=(224,224),
    batch_size=32
)

val_data.class_indices


In [ ]:
# Vizualization

t_img , label = train_data.next()

def plotImages(img_arr, label):
  """
  input  :- images array 
  output :- plots the images 
  """
  count = 0
  for im, l in zip(img_arr,label) :
    plt.imshow(im)
    plt.title(im.shape)
    plt.axis = False
    plt.show()
    
    count += 1
    if count == 10:
      break

plotImages(t_img, label)
     

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stopping
es = EarlyStopping(monitor='val_accuracy', min_delta= 0.01 , patience= 5, verbose= 1, mode='auto')

# model check point
mc = ModelCheckpoint(filepath="best_model.h5", monitor= 'val_accuracy', verbose= 1, save_best_only= True, mode = 'auto')

# puting call back in a list 
call_back = [es, mc]

In [ ]:
hist = model.fit_generator(train_data, 
                           steps_per_epoch= 10, 
                           epochs= 30, 
                           validation_data= val_data, 
                           validation_steps= 8, 
                           callbacks=[es,mc])